In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping

### 데이터 확인

In [7]:
pip install tensorflow

     ---------------------------------------- 0.0/48.7 kB ? eta -:--:--
     ------------------------------- ------ 41.0/48.7 kB 991.0 kB/s eta 0:00:01
     -------------------------------------- 48.7/48.7 kB 818.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 0.1/390.3 MB 812.7 kB/s eta 0:08:01
   ---------------------------------------- 0.1/390.3 MB 1.1 MB/s eta 0:05:50
   ---------------------------------------- 0.2/390.3 MB 1.2 MB/s eta 0:05:33
   ---------------------------------------- 0.3/390.3 MB 1.3 MB/s eta 0:04:58
   ---------------------------------------- 0.4/390.3 MB 1.5 MB/s eta 0:04:25
   ---------------------------------------- 0.5/390.3 MB 1.6 MB/s eta 0:04:12
   ---------------------------------------- 0.6/390.3 MB 1.7 MB/s eta 0:03:50
   ---------------------------------------- 0.7/390.3 MB 1.8 MB/s eta 0:03:39


### 모델 구축

In [15]:
train_a_path = 'train/TRAIN_A.csv'
train_b_path = 'train/TRAIN_B.csv'

train_a = pd.read_csv(train_a_path)
train_b = pd.read_csv(train_b_path)

In [21]:
train_data = pd.concat([train_a, train_b], ignore_index=True)
train_data

,timestamp,Q1,Q2,Q3,Q4,Q5,M1,M2,M3,M4,...,P17_flag,P18_flag,P19_flag,P20_flag,P21_flag,P22_flag,P23_flag,P24_flag,P25_flag,P26_flag
0,2024-05-27 0:00,17880.0000,37151.0010,24834.001,6321.0000,85828.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2024-05-27 0:01,17970.0000,37069.0010,25016.001,6367.0000,85212.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2024-05-27 0:02,17280.0000,37345.0000,24462.000,6431.0000,85655.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2024-05-27 0:03,17280.0000,37345.0000,24462.000,6431.0000,85619.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2024-05-27 0:04,17920.0000,37075.0000,24896.001,6206.0000,85619.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85856,24/07/29 23:55,28120.3125,6358.3604,13306.000,7869.9800,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85857,24/07/29 23:56,27784.5332,6382.7861,13325.000,7892.9087,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85858,24/07/29 23:57,27829.3750,6393.3931,13259.000,7892.9087,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85859,24/07/29 23:58,27705.7793,5732.1733,13536.000,7806.4023,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
feature_columns = [col for col in train_data.columns if col.startswith(('Q', 'P', 'M'))]
label_columns = [col for col in train_data.columns if col.startswith('P_flag')]

In [25]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_data[feature_columns] = scaler.fit_transform(train_data[feature_columns])

In [42]:
lookback = 3

In [40]:
def create_sequences(data, features, labels, lookback):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[features].iloc[i:i + lookback].values)
        y.append(data[labels].iloc[i + lookback].values)
    return np.array(X), np.array(y)

In [44]:
# Create sequences
X, y = create_sequences(train_data, feature_columns, label_columns, lookback)

KeyboardInterrupt: 

In [36]:
# Split into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y.reshape(-1))

KeyboardInterrupt: 

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32,
    callbacks=[early_stopping]
)

In [ ]:
# Save the trained model
model.save('trained_lstm_model.h5')
print("Model saved as 'trained_lstm_model.h5'")

In [ ]:
test_metadata_path = 'test.csv'
test_metadata = pd.read_csv(test_metadata_path)

In [ ]:
test_c_files = test_metadata[test_metadata['path'].str.contains('/C/')]['path'].tolist()
test_d_files = test_metadata[test_metadata['path'].str.contains('/D/')]['path'].tolist()

In [ ]:
def preprocess_test_data(test_file, feature_columns, lookback, scaler):
    """Preprocess individual test file for LSTM prediction."""
    test_data = pd.read_csv(test_file)
    test_data[feature_columns] = scaler.transform(test_data[feature_columns])  # Scale features
    X_test = []
    for i in range(len(test_data) - lookback):
        X_test.append(test_data[feature_columns].iloc[i:i + lookback].values)
    return np.array(X_test)

In [ ]:
predictions_c = []
for test_file in test_c_files:
    X_test_c = preprocess_test_data(test_file, feature_columns, lookback, scaler)
    pred_c = model.predict(X_test_c)
    predictions_c.append(pred_c)

In [ ]:
predictions_d = []
for test_file in test_d_files:
    X_test_d = preprocess_test_data(test_file, feature_columns, lookback, scaler)
    pred_d = model.predict(X_test_d)
    predictions_d.append(pred_d)

In [ ]:
def format_predictions(predictions, sensor_count):
    """Convert raw predictions into flag_list format."""
    flag_lists = []
    for pred in predictions:
        flag_lists.append('[' + ', '.join(map(str, (pred > 0.5).astype(int).flatten()[:sensor_count])) + ']')
    return flag_lists

In [ ]:
c_sensor_count = 10  # Replace with actual number of sensors in C
d_sensor_count = 8   # Replace with actual number of sensors in D

In [ ]:
flag_list_c = format_predictions(predictions_c, c_sensor_count)
flag_list_d = format_predictions(predictions_d, d_sensor_count)

In [ ]:
submission_c = pd.DataFrame({
    'ID': [f"TEST_C_{i}" for i in range(len(flag_list_c))],  # Replace with proper IDs from test.csv if available
    'flag_list': flag_list_c
})

submission_d = pd.DataFrame({
    'ID': [f"TEST_D_{i}" for i in range(len(flag_list_d))],  # Replace with proper IDs from test.csv if available
    'flag_list': flag_list_d
})

In [ ]:
# Combine C and D predictions
final_submission = pd.concat([submission_c, submission_d], ignore_index=True)

# Save submission
submission_path = 'sample_submission2.csv'
final_submission.to_csv(submission_path, index=False)
print("Submission file saved as 'sample_submission2.csv'")